In [1]:
import plotly.graph_objs as go
import networkx as nx
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
import pandas as pd
import numpy as np
import networkx as nx

In [41]:
# PATHs

## The directory where the dune data is stored
ORI_DATA_PATH = '../ori_data'

## The directory where the three databases are stored
DATABASE_PATH = '../database'

## The directory where the visualization data for each visualization is stored
VIS_DATA_PATH = '../../data'

## The directory where the scraped tweets data is stored
TWEET_PATH = '../ori_data/tweets'

In [42]:
# read transaction database
tx_db = pd.read_csv('{}/tx_db.csv'.format(DATABASE_PATH), index_col=0)
tx_db_2021 = pd.read_csv('{}/tx_db_2021.csv'.format(DATABASE_PATH), index_col=0)

# read cryptopunk database
punk_db = pd.read_csv('{}/punk_db.csv'.format(DATABASE_PATH), index_col=0)
punk_db['attributes'] = punk_db['attributes'].apply(eval)

# read trader database
trader_db = pd.read_csv('{}/trader_db.csv'.format(DATABASE_PATH), index_col=0)
trader_db['tx_involved'] = trader_db['tx_involved'].apply(eval)
trader_db_2021 = pd.read_csv('{}/trader_db_2021.csv'.format(DATABASE_PATH), index_col=0)
trader_db_2021['tx_involved'] = trader_db_2021['tx_involved'].apply(eval)

#### Create graph

In [43]:
tx_db_2021

,date_time,from,to,eth_price,punk_id,punk_skin_tone,punk_skin_tone_color
tx_id,,,,,,,
4940,2021-01-01 11:01:00,3610,1612,88.55,9368,Non-human,#8DFFCD
4941,2021-01-01 17:01:49,313,2036,11.44,4444,Dark,#A4031F
4942,2021-01-02 02:01:42,1122,1268,5.47,9340,Light,#F2A359
4943,2021-01-02 17:01:12,3421,4400,5.45,4861,Light,#F2A359
4944,2021-01-02 18:01:37,284,5818,5.30,9683,Dark,#A4031F
...,...,...,...,...,...,...,...
15407,2021-12-31 20:12:42,528,4049,103.00,6307,Albino,#F2DC5D
15408,2021-12-31 21:12:14,1546,5314,0.00,1482,Dark,#A4031F
15409,2021-12-31 23:12:16,4953,898,63.88,4085,Dark,#A4031F


In [44]:
trader_db_2021

,address,tx_involved
trader_id,,
0,\x20af8cccbce9013a9c0718046f49e1af9b238c08,"[7050, 6927, 6689, 6574, 6588, 6333, 6718, 659..."
1,\x19b5797670f84889b2d0d181ff208b8b2a190d2d,[12658]
2,\x3e9dacf58e308310e96b68924a57174f68a3d6cc,[10865]
3,\x78e6fe8e1d14a914c53b08e302e35443909dd24f,[10836]
4,\x5b4cad4874fdb0f2f641ddaa6fcbaced435b71f8,"[5245, 6399]"
...,...,...
5906,\xbd45481a84bc8edb066103c7dc9c62d53c275b12,"[14048, 12243, 12061]"
5907,\x4e7b8d5dfbef53faca35bbb2400785e93a6ef5aa,"[7351, 6189, 7375]"
5909,\xbda6ba3ad624582495c55451a1840e563fc16389,[14935]


In [54]:
nodes = get_nodes(trader_db_2021)
edges = get_edges(nodes, trader_db_2021, tx_db_2021)

In [59]:
# Create NetworkX Graph

def get_nodes(trader_db, degree_threshold=3):
    nodes = trader_db.copy(deep=True)
    nodes['degree'] = nodes['tx_involved'].apply(lambda x: len(x))
    nodes['node_id'] = nodes.index
    nodes = nodes.loc[:, ['node_id', 'degree']]
    nodes = nodes[nodes['degree'] > degree_threshold]
    nodes.sort_values(by='degree', ascending=False, inplace=True)
    return nodes


def get_edges(nodes, trader_db, tx_db):
    node_id_list = nodes['node_id'].tolist()
    edges = []
    for node_id in node_id_list:
        tx_involved = trader_db.loc[node_id, 'tx_involved']
        for tx in tx_involved:
            from_id = tx_db.loc[tx, 'from']
            to_id = tx_db.loc[tx, 'to']
            edges.append((from_id, to_id))
    return edges


def index_edge_skin_tone(txs):
    txs['from_to'] = txs.apply(lambda x: str(x['from']) + '_' + str(x['to']), axis=1)
    
    punk_skin_tone_dict = {}
    for index, row in txs.iterrows():
        from_id = row['from']
        to_id = row['to']
        skin_tone = row['punk_skin_tone']
        from_to = str(from_id) + '_' + str(to_id)
        to_from = str(to_id) + '_' + str(from_id)
        punk_skin_tone_dict[from_to] = skin_tone
        punk_skin_tone_dict[to_from] = skin_tone
    
    return punk_skin_tone_dict


def create_graph(traders, txs, degree_threshold=3):
    # nodes
    nodes = get_nodes(traders, degree_threshold=degree_threshold)
    
    # edges
    edges = get_edges(nodes, traders, txs)

    # init G
    G = nx.Graph()
    G.add_nodes_from(nodes['node_id'])
    G.add_edges_from(edges)
    
    # add pos attribute to nodes
    pos = nx.circular_layout(G)
    for node in G.nodes:
        G.nodes[node]['pos'] = list(pos[node])
        
    punk_skin_tone_dict = index_edge_skin_tone(txs)
    # add skin_tone attribute to links
    for edge in G.edges:
        from_id = edge[0]
        to_id = edge[1]
        from_to = str(from_id) + '_' + str(to_id)
        G.edges[edge]['skin_tone'] = punk_skin_tone_dict[from_to]
    
    return G


G = create_graph(trader_db_2021, tx_db_2021)
print('Number of nodes: {}'.format(len(G.nodes)))
print('Number of edges: {}'.format(len(G.edges)))

Number of nodes: 3971
Number of edges: 8963


In [67]:
# Use NetworkX Graph to create traces for visualization

def get_traces(G):
    # Node trace ----------------
    d = nx.degree(G)
    node_sizes = []
    for i in d:
        _, value = i
        node_sizes.append(value+5)
        
    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=dict(
            colorscale='Hot',
            reversescale=True,
            color=[],
            size=1,
            # colorbar=dict(
            #     thickness=15,
            #     title='Node Connections',
            #     xanchor='left',
            #     titleside='right'
            # ),
            line=dict(width=2)))

    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])

    for node, adjacencies in enumerate(G.adjacency()):
        node_trace['marker']['color'] += tuple([len(adjacencies[1])])
        node_info = 'Name: ' + \
            str(adjacencies[0]) + '<br># of connections: ' + \
            str(len(adjacencies[1]))
        node_trace['text'] += tuple([node_info])
        
    print('>>>>> node_trace created')

    # Edge traces ----------------
    edge_traces = dict()
    skin_tones = ['Medium', 'Dark', 'Light', 'Albino', 'Non-human']
    colors = ['#DB9065', '#A4031F', '#F2A359', '#F2DC5D', '#8DFFCD']
    color_by_skin_tone = dict(zip(skin_tones, colors))

    # edge trace with punk skin_tone == Light
    for skin_tone in skin_tones:
        edge_traces[skin_tone] = (go.Scatter(
            x=[],
            y=[],
            line=dict(width=0.3, color=color_by_skin_tone[skin_tone]),
            hoverinfo='none',
            mode='lines',
            name = skin_tone))
        
    print('>>>>> edge_traces dictionary initialized')

    for edge in G.edges():
        # line position
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']

        # skin tone
        skin_tone = G.edges[edge]['skin_tone']

        edge_traces[skin_tone]['x'] += tuple([x0, x1, None])
        edge_traces[skin_tone]['y'] += tuple([y0, y1, None])
        
    print('>>>>> edge_traces created')
    
    return node_trace, edge_traces



In [68]:
node_trace, edge_traces = get_traces(G)

>>>>> node_trace created
>>>>> edge_traces dictionary initialized
>>>>> edge_traces created


In [69]:
# Create Plotly figure

def create_fig(node_trace, edge_traces):
    edge_traces_list = [val for val in edge_traces.values()]
    edge_traces_list.append(node_trace)
    edge_traces_list
    
    # Figure
    traces = [val for val in edge_traces.values()].append(node_trace)
    f = go.Figure(data=edge_traces_list,
                    layout=go.Layout(
                        title='<br>Network Graph of CryptoPunks Transactions in 2021',
                        titlefont=dict(size=16),
                        showlegend=True,
                        hovermode='closest',
                        width=880,
                        height=800,
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=[dict(
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002)],
                        xaxis=dict(showgrid=False, zeroline=False,
                                    showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False,
                                    showticklabels=False)
                    )
                )
    
    return f
    print('>>>>> fig created')

In [70]:
fig = create_fig(node_trace, edge_traces)

In [71]:
iplot(fig)